# Calculate cloud costs per day


In [4]:
import json
import subprocess
import csv
import sys
from google.cloud import storage
import io
import tempfile
from dateutil.parser import parse

In [5]:
def totals_from_json(file):
    totals = {}
    for item in json.load(file):
        cost = float(item['cost']['amount'])
        time_range = (item['startTime'], item['endTime'])
        totals[time_range] = totals.get(time_range, 0) + cost

    return totals

def totals_from_csv(file):
    totals = {}
    reader = csv.DictReader(file)
    for row in reader:
        time_range = (row['Start Time'], row['End Time'])
        totals[time_range] = totals.get(time_range, 0) + float(row['Cost'])

    return totals

In [8]:
def get_daily_cost(project, billing_bucket_name, kind='json'):
    totals = {}
    client = storage.Client(project)

    bucket = storage.Bucket(client, billing_bucket_name)
    if kind == 'csv':
        prefix='report-'
    else:
        prefix='billing-'
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        buffer = io.StringIO(blob.download_as_string().decode())

        if kind == 'csv':
            current_totals = totals_from_csv(buffer)
        else:
            current_totals = totals_from_json(buffer)

        for time_range, cost in current_totals.items():
            totals[time_range] = totals.get(time_range, 0) + cost


    # We want to push out sorted jsonl
    sorted_items = [
        { 'startTime': start_time, 'endTime': end_time, 'cost': cost }
        for (start_time, end_time), cost in totals.items()
    ]

    sorted_items.sort(key=lambda d: d['startTime'])

    return sorted_items

In [9]:
cost = get_daily_cost('ucb-datahub-2018','ucb_datahub_18_billing')

/home/yuvipanda/.local/share/virtualenvs/berkeley-datahub-analysis-rzfKRwiq/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [13]:
with open('../data/processed/cloud-costs.jsonl', 'w') as f:
    json.dump(cost, f)